In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/thaijo-researcher-for-code-submission/sample_submission.csv
/kaggle/input/thaijo-researcher-for-code-submission/test.csv
/kaggle/input/thaijo-researcher-for-code-submission/thaijo_cleansing.py


In [2]:
import numpy as np
import pandas as pd
import regex as re

dir = '/kaggle/input/thaijo-researcher-for-code-submission/'
input_df = pd.read_csv(dir+'test.csv')
input_df

,Unnamed: 0,_id,_source.author,_source.co-author
0,1062,8449,ดวงรัตน์ วัฒนกิจไกรเลิศ,"ดวงรัตน์ วัฒนกิจไกรเลิศ, ละเอียด จารุสมบัติ, ว..."
1,1063,8485,นิตญา ฤทธิ์เพชร,"นิตญา ฤทธิ์เพชร, ชนกพร จิตปัญญา"
2,1064,8490,สุจิตราภรณ์ พิมพ์โพธิ์,"สุจิตราภรณ์ พิมพ์โพธิ์, ชนกพร จิตปัญญา"
3,1065,8507,ปรวรรรณ วิทย์วรานุกูล,"ปรวรรรณ วิทย์วรานุกูล, ชนกพร จิตปัญญา"
4,1066,8460,ชนกพร จิตปัญญา,"ชนกพร จิตปัญญา, พูลสุข หิรัญสาย, สุชาต ไชยโรจน์"
...,...,...,...,...
4725,15920,140582,สถิตย์พงษ์ วงศ์สง่า,"สถิตย์พงษ์ วงศ์สง่า, อาภา สธนเสาวภาคย์"
4726,15921,142099,Pornwalai Boonmuang,"Pornwalai Boonmuang, Wichai Santimaleeworagun,..."
4727,15922,140493,กองบรรณาธิการ วารสาร,กองบรรณาธิการ วารสาร
4728,15923,140479,Wanna Viriyathavekul,Wanna Viriyathavekul


In [3]:
input_df[['_id','_source.co-author']].iloc[1500:1550]

,_id,_source.co-author
1500,123790,พิโรดม พิริยพฤทธิ์
1501,123171,ปิยะภัสร์ โรจน์รัตนวาณิชย์
1502,123309,ปรมาภรณ์ พูลสวัสดิ์
1503,150470,ชวลิต จิตต์ชื่น
1504,151836,"สิทธิพงษ์ ศิริประทุม, ชัจคเณค์ แพรขาว"
1505,151827,"นภัสภรณ์ เชิงสะอาด, เทอดศักดิ์ พรหมอารักษ์, วั..."
1506,150318,Nawin Kuntaraksa
1507,150394,ผศ.ดร.แสวง แสนบุตร
1508,150338,บุษกร จินต์ธนาวัฒน์
1509,151848,"ปิ่นเพชร อำภรณ์, ประภัสสร สมศรี, ทัศนีย์ แดขุน..."


# EDA

In [4]:
# check NaN
input_df[input_df['_source.co-author'].isna()]
#len(input_df[input_df['_source.co-author'].isna()])

,Unnamed: 0,_id,_source.author,_source.co-author
805,81368,48075,ณัฐนัย ศรีโรจน์,NaN
806,81369,48070,กลิ่นผกา ทับทิมธงไชย,NaN
807,81370,48083,วิระ บำรุงศรี,NaN
808,81371,48090,อรัญ วานิชกร,NaN
3174,5936,2521,กองบรรณาธิการ วารสารสถาบันวัฒนธรรมและศิลปะ,NaN
3176,5938,2522,ศ.ดร.วีรชาติ เปรมานนท์,NaN
3177,5939,2516,สุภัค มหาวรากร,NaN
3180,5942,2511,วิภาดา เพชรโชติ,NaN
3675,14725,105630,มณีรัตน์ สุขเกษม,NaN
4030,15155,105622,ฌานิก หวังพานิช,NaN


In [5]:
# filter NaN
filtered_df = input_df.fillna('')
filtered_df[filtered_df['_source.co-author'].isna()]

,Unnamed: 0,_id,_source.author,_source.co-author


In [6]:
# case 1: remove prefixes
def remove_prefixes(text):
    remove_pattern_en = r'(?:Authors\s*:|Author\s*:|\,*\s*M\.R\.|M\.L\.|MRS\.|Mrs\.|Mr\.|Ms\.|MR\.|MS\.|M\.D\.|Dr\.|Assoc\.|Asst\.|Assist\.*|Prof\.|Lect\.|Ven\.|^\(| Authors :)\s*'
    removed_t = re.sub(remove_pattern_en, '', text)
    remove_pattern_th = r'(?:ว่าที่ร้อยตรี|ม\.ร\.ว\.|ว่าที่ พ\.ต\.|ร\.ต\.อ.|ว่าที่ ร\.ต\.|นพ\.|ดร\.|ผู้ช่วยศาสตราจารย์|รองศาสตราจารย์|ผศ\.|รศ\.|อ\.|อาจารย์|พระปลัด|พระมหา|พระครู|พระ|กสทช\.)\s*'
    removed_t = re.sub(remove_pattern_th, '', removed_t)
    return removed_t

pref_filtered_df = filtered_df.copy()
pref_filtered_df['_source.author']= pref_filtered_df['_source.author'].apply(remove_prefixes)
pref_filtered_df['_source.co-author']= pref_filtered_df['_source.co-author'].apply(remove_prefixes)
pref_filtered_df

,Unnamed: 0,_id,_source.author,_source.co-author
0,1062,8449,ดวงรัตน์ วัฒนกิจไกรเลิศ,"ดวงรัตน์ วัฒนกิจไกรเลิศ, ละเอียด จารุสมบัติ, ว..."
1,1063,8485,นิตญา ฤทธิ์เพชร,"นิตญา ฤทธิ์เพชร, ชนกพร จิตปัญญา"
2,1064,8490,สุจิตราภรณ์ พิมพ์โพธิ์,"สุจิตราภรณ์ พิมพ์โพธิ์, ชนกพร จิตปัญญา"
3,1065,8507,ปรวรรรณ วิทย์วรานุกูล,"ปรวรรรณ วิทย์วรานุกูล, ชนกพร จิตปัญญา"
4,1066,8460,ชนกพร จิตปัญญา,"ชนกพร จิตปัญญา, พูลสุข หิรัญสาย, สุชาต ไชยโรจน์"
...,...,...,...,...
4725,15920,140582,สถิตย์พงษ์ วงศ์สง่า,"สถิตย์พงษ์ วงศ์สง่า, อาภา สธนเสาวภาคย์"
4726,15921,142099,Pornwalai Boonmuang,"Pornwalai Boonmuang, Wichai Santimaleeworagun,..."
4727,15922,140493,กองบรรณาธิการ วารสาร,กองบรรณาธิการ วารสาร
4728,15923,140479,Wanna Viriyathavekul,Wanna Viriyathavekul


In [7]:
# case 2: remove subfixes
def remove_subfixes(text):
    remove_pattern = 'คณะนักศึกษา.*|\s*and Other|\s*And Other|\s*และคณะ|\,*\s*et\.*\s*al\.*|\*$|\,\s*M\.R\.|M\s*R\.|\s+\.$|\s+\-|\,|\s*CFA'
    # Splitting the text into names, cleaning each, and then rejoining
    cleaned_names = [re.sub(remove_pattern, '', name).strip() for name in text.split(', ')]
    # Removing duplicates and rejoining into a single string
    unique_cleaned_names = list(dict.fromkeys(cleaned_names))
    return ','.join(unique_cleaned_names)

def remove_comma_last(text):
    remove_pattern = '\,\s*$'
    # Splitting the text into names, cleaning each, and then rejoining
    cleaned_names = [re.sub(remove_pattern, '', name).strip() for name in text.split(', ')]
    # Removing duplicates and rejoining into a single string
    unique_cleaned_names = list(dict.fromkeys(cleaned_names))
    return ','.join(unique_cleaned_names) 

subf_filtered_df = pref_filtered_df.copy()
# Apply the updated remove_subfixes function to each name in the specified columns
subf_filtered_df['_source.author'] = subf_filtered_df['_source.author'].apply(remove_subfixes)
subf_filtered_df['_source.co-author'] = subf_filtered_df['_source.co-author'].apply(remove_subfixes)
subf_filtered_df['_source.author'] = subf_filtered_df['_source.author'].apply(remove_comma_last)
subf_filtered_df['_source.co-author'] = subf_filtered_df['_source.co-author'].apply(remove_comma_last)
subf_filtered_df.iloc[510:515]

,Unnamed: 0,_id,_source.author,_source.co-author
510,71134,38105,เจริญพงศ์ พรหมศร,เจริญพงศ์ พรหมศร
511,71135,38124,กิตติพัฒน์ อุโฆษกิจ,"กิตติพัฒน์ อุโฆษกิจ,มาริสา รักสุขสมบัติ,สุภาภร..."
512,71136,38261,สุวัฒน์ เปี่ยมปัจจัย,"สุวัฒน์ เปี่ยมปัจจัย,ดวงตา ตันโช"
513,71137,38260,จันทวรรณ สุจริตกุล,จันทวรรณ สุจริตกุล
514,71138,38257,สุรสิทธิ์ ถนัดทาง,สุรสิทธิ์ ถนัดทาง


In [8]:
# case 3: remove ' - ', ' 1 '
def remove_mid(text):
    remove_pattern = '\s+\-\s+|\s+\d+\s+'
    return re.sub(remove_pattern, ' ', text)

mid_filtered_df = subf_filtered_df.copy()
mid_filtered_df['_source.author']= mid_filtered_df['_source.author'].apply(remove_mid)
mid_filtered_df['_source.co-author']= mid_filtered_df['_source.co-author'].apply(remove_mid)
mid_filtered_df

,Unnamed: 0,_id,_source.author,_source.co-author
0,1062,8449,ดวงรัตน์ วัฒนกิจไกรเลิศ,"ดวงรัตน์ วัฒนกิจไกรเลิศ,ละเอียด จารุสมบัติ,วัน..."
1,1063,8485,นิตญา ฤทธิ์เพชร,"นิตญา ฤทธิ์เพชร,ชนกพร จิตปัญญา"
2,1064,8490,สุจิตราภรณ์ พิมพ์โพธิ์,"สุจิตราภรณ์ พิมพ์โพธิ์,ชนกพร จิตปัญญา"
3,1065,8507,ปรวรรรณ วิทย์วรานุกูล,"ปรวรรรณ วิทย์วรานุกูล,ชนกพร จิตปัญญา"
4,1066,8460,ชนกพร จิตปัญญา,"ชนกพร จิตปัญญา,พูลสุข หิรัญสาย,สุชาต ไชยโรจน์"
...,...,...,...,...
4725,15920,140582,สถิตย์พงษ์ วงศ์สง่า,"สถิตย์พงษ์ วงศ์สง่า,อาภา สธนเสาวภาคย์"
4726,15921,142099,Pornwalai Boonmuang,"Pornwalai Boonmuang,Wichai Santimaleeworagun,W..."
4727,15922,140493,กองบรรณาธิการ วารสาร,กองบรรณาธิการ วารสาร
4728,15923,140479,Wanna Viriyathavekul,Wanna Viriyathavekul


In [9]:
# case 4 : remove_strings_with_keywords
def remove_strings_with_keywords(input_string):
    keywords = ['บรรณาธิการ', 'Journal', 'วิทยาลัย', 'ผู้ทรงคุณวุฒิ','^\s*\-\s+\-',  '^\s*\---\s*', '^\s*\--\s+\--', '^\s*\.\s+\.','^\s*\-\s*',  'subcommittee', 'Editorial', 'สภากาชาดไทย', 'แนะนำผู้เขียน', 'Thailand', 'JPHD', 'Vol\.']
    for keyword in keywords:
        if re.search(keyword, input_string, flags=re.IGNORECASE):
            return ''
    return input_string

kw_filtered_df = mid_filtered_df.copy()
kw_filtered_df['_source.author']= kw_filtered_df['_source.author'].apply(remove_strings_with_keywords)
kw_filtered_df['_source.co-author']= kw_filtered_df['_source.co-author'].apply(remove_strings_with_keywords)

kw_filtered_df[kw_filtered_df['_id']=='63998']

,Unnamed: 0,_id,_source.author,_source.co-author
3125,5887,63998,รศรินทร เกรย,"รศรินทร เกรย,สาลินี เทพสุวรรณ"


In [10]:
# case 5: Replace typo eg. ';' with ',' | '((' with '(' |  ') ก' as '),'
def replace_typo(text):
    # First, replace all semicolons with commas
    text = re.sub(';', ',', text)
    
    # Next, replace all double open parentheses with a single open parenthesis
    text = re.sub('\(\(', '(', text)

    # replace non-ASCII character followed by ASCII letters
    text = re.sub('[ก-๙](?=[a-zA-Z])','', text)

    # # replace ') ก' as '),'
    # text = re.sub('\)\s(?![,])', '),', text)

    # replace number with blank
    text = re.sub('\d+|', '', text)
    
    # remove '.' with blank
    text = re.sub('^\s*\.+\s*', '',text)

    return text

typo_filtered_df = kw_filtered_df.copy()
typo_filtered_df['_source.author']= typo_filtered_df['_source.author'].apply(replace_typo)
typo_filtered_df['_source.co-author']= typo_filtered_df['_source.co-author'].apply(replace_typo)
typo_filtered_df

,Unnamed: 0,_id,_source.author,_source.co-author
0,1062,8449,ดวงรัตน์ วัฒนกิจไกรเลิศ,"ดวงรัตน์ วัฒนกิจไกรเลิศ,ละเอียด จารุสมบัติ,วัน..."
1,1063,8485,นิตญา ฤทธิ์เพชร,"นิตญา ฤทธิ์เพชร,ชนกพร จิตปัญญา"
2,1064,8490,สุจิตราภรณ์ พิมพ์โพธิ์,"สุจิตราภรณ์ พิมพ์โพธิ์,ชนกพร จิตปัญญา"
3,1065,8507,ปรวรรรณ วิทย์วรานุกูล,"ปรวรรรณ วิทย์วรานุกูล,ชนกพร จิตปัญญา"
4,1066,8460,ชนกพร จิตปัญญา,"ชนกพร จิตปัญญา,พูลสุข หิรัญสาย,สุชาต ไชยโรจน์"
...,...,...,...,...
4725,15920,140582,สถิตย์พงษ์ วงศ์สง่า,"สถิตย์พงษ์ วงศ์สง่า,อาภา สธนเสาวภาคย์"
4726,15921,142099,Pornwalai Boonmuang,"Pornwalai Boonmuang,Wichai Santimaleeworagun,W..."
4727,15922,140493,,
4728,15923,140479,Wanna Viriyathavekul,Wanna Viriyathavekul


In [11]:
# extract Thai Eng

def extract_th_en_ids(txt):
    if re.search(r'[ก-๙]+\s+[ก-๙]+\s+\(*\s*[A-Za-z]+\s+[A-Za-z]+\s*\)*', txt): #มนตรี ศรีราชพัฒน์ (Montri Srirachapat)
        return re.sub(r'([ก-๙]+\s+[ก-๙]+)\s+\(*\s*([A-Za-z]+)\s+([A-Za-z]+)\s*\)*',lambda m: m.group(1)+','+ m.group(2)+' '+m.group(3), txt)
    elif re.search(r'[ก-๙]+\s*\(\s*[A-Za-z]+\s*\)\s*[ก-๙]+\s*\(\s*[A-Za-z]+\s*\)', txt): #ดุจเดือน (Duchduen) พันธุมนาวิน (Bhanthumnavin)
        return re.sub(r'([ก-๙]+)\s*\(\s*([A-Za-z]+)\s*\)\s*([ก-๙]+)\s*\(\s*([A-Za-z]+)\s*\)',lambda m: m.group(1)+' ' + m.group(3)+',' + m.group(2)+' '+ m.group(4), txt)
    else:
        return txt
    
th_en_ext_df = typo_filtered_df.copy()
th_en_ext_df['_source.author']= th_en_ext_df['_source.author'].apply(extract_th_en_ids)
th_en_ext_df['_source.co-author']= th_en_ext_df['_source.co-author'].apply(extract_th_en_ids)
th_en_ext_df
extract_th_en_ids('มนตรี ศรีราชพัฒน์ Montri Srirachapat,ศรัณยา เลิศพุทธรักษ์ (Sarunya Lertputtarak)')
extract_th_en_ids('ดุจเดือน (Duchduen) พันธุมนาวิน (Bhanthumnavin)')
# extract_th_en_ids('สมพร (Somporn) สุทัศนีย์ (Sudhasani),เสรี (Seree) ชัดแช้ม(Chadcham)')

'ดุจเดือน พันธุมนาวิน,Duchduen Bhanthumnavin'

In [12]:
# Uppercases 
def process_word(word):
    # Handling for hyphenated names
    # if '-' in word:
    #     parts = word.split('-')
    #     processed_parts = [part.capitalize() for part in parts]
    #     return '-'.join(processed_parts)
    # Handling for names with apostrophes like O'connell
    if '’' in word or "'" in word:
        # print('A')
        parts = re.split(r"(’|')", word)  # Split by apostrophe but keep it for rejoining
        processed_parts = [parts[0].capitalize()] + parts[1:]  # Capitalize before the apostrophe
        for i in range(2, len(parts), 2):  # Capitalize after the apostrophe
            processed_parts[i] = processed_parts[i].capitalize()
        return ''.join(processed_parts)
    # Special handling for initials and abbreviations
    elif re.search(r'\b[A-Za-z]\.\b', word):
        return re.sub(r'\b([A-Za-z])\.\,?', lambda m: m.group(1).upper() + '.', word)
    elif word.isupper():
        # print('C')
        return word.capitalize()
    # Capitalize the first letter, lowercase the rest for other words
    elif word[0].isalpha() and word[0].islower():
        # print('D')
        return word.capitalize()
    return word

def capitalize_english_text(text):
    # Process each word separately
    new_name = []
    for name in text.split(','):
        new_word = []
        for word in name.split():
            new_word.append(process_word(word))
        new_name.append(' '.join(new_word))
    return ', '.join(new_name)

def process_text(text):
    # Leave text inside parentheses as is if it's already correctly capitalized
    def capitalize_within_brackets(match):
        inner_text = match.group(1)
        capitalized_text = capitalize_english_text(inner_text)
        return match.group(0)[0] + capitalized_text + match.group(0)[-1]
    
    # Apply capitalization rules within parentheses and square brackets
    text = re.sub(r'\(([^)]+)\)', capitalize_within_brackets, text)
    text = re.sub(r'\[([^]]+)\]', capitalize_within_brackets, text)
    
    # Process and capitalize the rest of the text
    return capitalize_english_text(text)

case_filtered_df = th_en_ext_df.copy()
case_filtered_df['_source.author']= case_filtered_df['_source.author'].apply(process_text)
case_filtered_df['_source.co-author']= case_filtered_df['_source.co-author'].apply(process_text)
case_filtered_df

,Unnamed: 0,_id,_source.author,_source.co-author
0,1062,8449,ดวงรัตน์ วัฒนกิจไกรเลิศ,"ดวงรัตน์ วัฒนกิจไกรเลิศ, ละเอียด จารุสมบัติ, ว..."
1,1063,8485,นิตญา ฤทธิ์เพชร,"นิตญา ฤทธิ์เพชร, ชนกพร จิตปัญญา"
2,1064,8490,สุจิตราภรณ์ พิมพ์โพธิ์,"สุจิตราภรณ์ พิมพ์โพธิ์, ชนกพร จิตปัญญา"
3,1065,8507,ปรวรรรณ วิทย์วรานุกูล,"ปรวรรรณ วิทย์วรานุกูล, ชนกพร จิตปัญญา"
4,1066,8460,ชนกพร จิตปัญญา,"ชนกพร จิตปัญญา, พูลสุข หิรัญสาย, สุชาต ไชยโรจน์"
...,...,...,...,...
4725,15920,140582,สถิตย์พงษ์ วงศ์สง่า,"สถิตย์พงษ์ วงศ์สง่า, อาภา สธนเสาวภาคย์"
4726,15921,142099,Pornwalai Boonmuang,"Pornwalai Boonmuang, Wichai Santimaleeworagun,..."
4727,15922,140493,,
4728,15923,140479,Wanna Viriyathavekul,Wanna Viriyathavekul


In [13]:
# Test with the provided examples
test_texts = [
    'Jirawat TIJAN,Sainatee PRATANAPHON,Peanchai Khamwong',
    'Nikhade G.R.,Chiddarwar S.S.,Deshpande V.S.',

    'M.N. Bargach',
    'ชาริณี กีรติโชติ (Charinee Keeratichote)',
    'ชาริณี กีรติโชติ [Charinee Keeratichote]',
    'Amorn SUWANNIMITR',
    'Nikhade G.r.',
    'LE-THANH Nhan',
    'KEN-ICHIRO HISADA',
    'Uthen LOUG-UI',
    'Uthen Loug-Ui',
    'Prapaisri Sudasna-na-Ayudthya',
    'Ninthana Iamsa-ard',
    'Itsari Srisupha-olarn, Noraluk Ua-Kit',
    'Katharine O’connell',
    'Jirawat TIJAN'

]
for test_text in test_texts:
    processed_text = process_text(test_text)
    print(f"Original: '{test_text}'\nProcessed: '{processed_text}'\n")

Original: 'Jirawat TIJAN,Sainatee PRATANAPHON,Peanchai Khamwong'
Processed: 'Jirawat Tijan, Sainatee Pratanaphon, Peanchai Khamwong'

Original: 'Nikhade G.R.,Chiddarwar S.S.,Deshpande V.S.'
Processed: 'Nikhade G.R., Chiddarwar S.S., Deshpande V.S.'

Original: 'M.N. Bargach'
Processed: 'M.N. Bargach'

Original: 'ชาริณี กีรติโชติ (Charinee Keeratichote)'
Processed: 'ชาริณี กีรติโชติ (Charinee Keeratichote)'

Original: 'ชาริณี กีรติโชติ [Charinee Keeratichote]'
Processed: 'ชาริณี กีรติโชติ [Charinee Keeratichote]'

Original: 'Amorn SUWANNIMITR'
Processed: 'Amorn Suwannimitr'

Original: 'Nikhade G.r.'
Processed: 'Nikhade G.R.'

Original: 'LE-THANH Nhan'
Processed: 'Le-thanh Nhan'

Original: 'KEN-ICHIRO HISADA'
Processed: 'Ken-ichiro Hisada'

Original: 'Uthen LOUG-UI'
Processed: 'Uthen Loug-ui'

Original: 'Uthen Loug-Ui'
Processed: 'Uthen Loug-Ui'

Original: 'Prapaisri Sudasna-na-Ayudthya'
Processed: 'Prapaisri Sudasna-na-Ayudthya'

Original: 'Ninthana Iamsa-ard'
Processed: 'Ninthana Iamsa-

In [14]:
#'name'
df = case_filtered_df.copy()
df['name'] = df['_source.co-author']
df

,Unnamed: 0,_id,_source.author,_source.co-author,name
0,1062,8449,ดวงรัตน์ วัฒนกิจไกรเลิศ,"ดวงรัตน์ วัฒนกิจไกรเลิศ, ละเอียด จารุสมบัติ, ว...","ดวงรัตน์ วัฒนกิจไกรเลิศ, ละเอียด จารุสมบัติ, ว..."
1,1063,8485,นิตญา ฤทธิ์เพชร,"นิตญา ฤทธิ์เพชร, ชนกพร จิตปัญญา","นิตญา ฤทธิ์เพชร, ชนกพร จิตปัญญา"
2,1064,8490,สุจิตราภรณ์ พิมพ์โพธิ์,"สุจิตราภรณ์ พิมพ์โพธิ์, ชนกพร จิตปัญญา","สุจิตราภรณ์ พิมพ์โพธิ์, ชนกพร จิตปัญญา"
3,1065,8507,ปรวรรรณ วิทย์วรานุกูล,"ปรวรรรณ วิทย์วรานุกูล, ชนกพร จิตปัญญา","ปรวรรรณ วิทย์วรานุกูล, ชนกพร จิตปัญญา"
4,1066,8460,ชนกพร จิตปัญญา,"ชนกพร จิตปัญญา, พูลสุข หิรัญสาย, สุชาต ไชยโรจน์","ชนกพร จิตปัญญา, พูลสุข หิรัญสาย, สุชาต ไชยโรจน์"
...,...,...,...,...,...
4725,15920,140582,สถิตย์พงษ์ วงศ์สง่า,"สถิตย์พงษ์ วงศ์สง่า, อาภา สธนเสาวภาคย์","สถิตย์พงษ์ วงศ์สง่า, อาภา สธนเสาวภาคย์"
4726,15921,142099,Pornwalai Boonmuang,"Pornwalai Boonmuang, Wichai Santimaleeworagun,...","Pornwalai Boonmuang, Wichai Santimaleeworagun,..."
4727,15922,140493,,,
4728,15923,140479,Wanna Viriyathavekul,Wanna Viriyathavekul,Wanna Viriyathavekul


In [15]:
def remove_spaces(string):
    if string == '':
        return ''
    elif string == ' ':
        return ''
    if string[0] == ' ':
        string = string[1:]
    if string[-1] == ' ':
        string = string[:-1]
    return string

result_new = {'id': [], 'name': []}
for idx, row in df.iterrows():
    names = []
    result_new['id'].extend([f"{row['_id']}_{i+1}" for i in range(10)])
    # author = row['_source.author']
    # if author != '':
    #     names.append(author)
    author = row['_source.author']
    if author != '':
        ca = author.split(',')
        names.append(ca[0])   
    co_author = row['name']
    if co_author != '':
        ca = co_author.split(',')
        for c in ca:
            if remove_spaces(c) not in names:
                names.append(c)
    if len(names) == 0:
        result_new['name'].extend([None]*10)
        continue
    names = [remove_spaces(name) for name in names]
    if len(names) > 10:
        names = names[:10]
    elif len(names) < 10:
        names.extend(['']*(10-len(names)))
    result_new['name'].extend(names)
result_new = pd.DataFrame(result_new)
result_new.to_csv('/kaggle/working/code_submission.csv', index=False)

In [16]:
result_new[result_new['id']=='63998_1']

,id,name
31250,63998_1,รศรินทร เกรย
